In [1]:
# -------- 1.a: Instrument the original to SHOW what's wrong (via print only) --------
def wrong_add_function(arg1, arg2):
    '''
    The function takes in two lists of integers, then it adds
    all of arg2 to each item of arg1.

    Example (what it CURRENTLY does, incorrectly):
       > wrong_add_function([1,2,3],[1,1,1])  -> [6,9,12]

    Whereas the expected correct answer is [2,3,4] (pairwise addition).
    '''
    arg1_index = 0
    while arg1_index < len(arg1):
        # Explain the intent and expected value for THIS index:
        if arg1_index < len(arg2):
            expected_here = arg1[arg1_index] + arg2[arg1_index]
        else:
            expected_here = arg1[arg1_index]  # graceful fallback if lens mismatch

        print(f"[DEBUG] About to process index {arg1_index}: "
              f"arg1[{arg1_index}]={arg1[arg1_index]}, arg2={arg2}")
        print(f"We are making an error in the loop at index {arg1_index}. "
              f"The correct answer is supposed to be: {expected_here}")

        # Show what the buggy loop is ABOUT to compute:
        preview = [arg1[arg1_index] + i for i in arg2]
        print(f"But this loop will instead sum these values: {preview} "
              f"-> wrong total = {sum(preview)}")

        # --- BUGGY CODE (kept as-is, just with prints around it) ---
        arg_2_sum = 0
        for arg2_elements in arg2:
            arg_2_sum = sum([arg1[arg1_index] + i for i in arg2])
        arg1[arg1_index] = arg_2_sum
        # -----------------------------------------------------------

        print(f"[DEBUG] After buggy computation at index {arg1_index}: "
              f"arg1[{arg1_index}]={arg1[arg1_index]}\n")
        arg1_index += 1
    return arg1


# -------- 1.b: Minimal-change correct version (same general structure) --------
def correct_add_function(arg1, arg2):
    '''
    Correct behavior: pairwise addition.
    Example:
      correct_add_function([1,2,3],[1,1,1]) -> [2,3,4]
    '''
    arg1_index = 0
    while arg1_index < len(arg1):
        # Keep the same overall structure, but only add the matching element.
        # If arg2 is shorter, treat the missing as 0 (keeps function tolerant).
        addend = arg2[arg1_index] if arg1_index < len(arg2) else 0

        # (Optional) clarity print showing the correct per-index expectation
        print(f"[INFO] Computing index {arg1_index}: "
              f"{arg1[arg1_index]} + {addend} -> {arg1[arg1_index] + addend}")

        # Keep "for" loop shell to change as little as possible:
        arg_2_sum = 0
        for j, arg2_elements in enumerate(arg2):
            if j == arg1_index:
                arg_2_sum = arg1[arg1_index] + arg2_elements
                break
        else:
            # If we never hit the matching index in arg2 (e.g., arg2 shorter),
            # fall back to adding 0.
            arg_2_sum = arg1[arg1_index] + 0

        arg1[arg1_index] = arg_2_sum
        arg1_index += 1
    return arg1


# =============================================================================
# 2.a/2.b/2.c
# Use the corrected NUMERIC section from (1.b) for the functions below.
# =============================================================================

# Helper predicates
def _all_ints(lst):  return all(isinstance(x, int) for x in lst)
def _all_strs(lst):  return all(isinstance(x, str) for x in lst)


# -------- 2.b: Return a friendly error when input types are invalid --------
def exception_add_function(arg1, arg2):
    '''
    Uses the corrected numeric logic from (1.b) and the original string logic.
    If either list is supposed to be numeric but contains a bad type,
    return: "Your input argument [1 or 2] at element [n] is not of the expected type. Please change this and rerun."
    '''
    try:
        # Case 1: both numeric -> corrected numeric section
        if _all_ints(arg1) and _all_ints(arg2):
            # corrected numeric section
            arg1_index = 0
            while arg1_index < len(arg1):
                addend = arg2[arg1_index] if arg1_index < len(arg2) else 0
                # keep minimal structure:
                arg_2_sum = 0
                for j, arg2_elements in enumerate(arg2):
                    if j == arg1_index:
                        arg_2_sum = arg1[arg1_index] + arg2_elements
                        break
                else:
                    arg_2_sum = arg1[arg1_index] + 0
                arg1[arg1_index] = arg_2_sum
                arg1_index += 1
            return arg1

        # Case 2: both strings -> original string section
        if _all_strs(arg1) and _all_strs(arg2):
            arg1_index = 0
            while arg1_index < len(arg1):
                arg_2_sum = ''
                for arg2_elements in arg2:
                    arg_2_sum += arg2_elements
                arg1[arg1_index] = arg1[arg1_index] + str(arg_2_sum)
                arg1_index += 1
            return arg1

        # Otherwise, find the first offending spot and return the required message
        # Decide which mode user likely intended: numeric or string.
        intend_numeric = any(isinstance(x, int) for x in arg1 + arg2)
        if intend_numeric:
            # Find first non-int in arg1, then arg2
            for n, v in enumerate(arg1):
                if not isinstance(v, int):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, int):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."
        else:
            # Intended strings: find first non-str
            for n, v in enumerate(arg1):
                if not isinstance(v, str):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, str):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."

        # Fallback (shouldn't reach)
        return "Invalid inputs."
    except Exception:
        # Generic safety net (kept simple as requested)
        return "An unexpected error occurred while processing your inputs."


# -------- 2.c: Coerce mixed string/int inputs so STRING section runs, without changing wrong_add_function --------
def correction_add_function(arg1, arg2):
    '''
    Do NOT modify wrong_add_function.
    Goal: if inputs are intended as strings but contain ints, coerce within try/except
    so wrong_add_function’s *string* section runs unmodified.
    '''
    try:
        # If either list appears to be "string-like" (has at least one str),
        # coerce BOTH lists to strings so the original string section will handle them.
        if any(isinstance(x, str) for x in arg1 + arg2):
            arg1_coerced = [str(x) for x in arg1]
            arg2_coerced = [str(x) for x in arg2]
            return wrong_add_function(arg1_coerced, arg2_coerced)

        # Otherwise, leave them as-is and just call wrong_add_function (numeric case).
        return wrong_add_function(arg1, arg2)
    except Exception:
        return "Could not coerce inputs for string processing."
print("Hello world")

Hello world


In [1]:
# ----------------------------------------------------------
# 1.a — Instrumented buggy version (prints where/why it errs)
# ----------------------------------------------------------
def wrong_add_function(arg1, arg2):
    """
    Demonstrates where the loop logic goes wrong.
    Expected: pairwise addition.
    Current: adds all elements of arg2 to each element of arg1.
    """
    arg1_index = 0
    while arg1_index < len(arg1):
        expected = arg1[arg1_index] + (arg2[arg1_index] if arg1_index < len(arg2) else 0)
        print(f"We are making an error in the loop at index {arg1_index}. "
              f"The correct answer is supposed to be: {expected}")
        preview = [arg1[arg1_index] + i for i in arg2]
        print(f"[DEBUG] Buggy will sum {preview} -> {sum(preview)}")
        # keep buggy code intact
        arg_2_sum = 0
        for _ in arg2:
            arg_2_sum = sum([arg1[arg1_index] + i for i in arg2])
        arg1[arg1_index] = arg_2_sum
        arg1_index += 1
    return arg1


# ----------------------------------------------------------
# 1.b — Minimal-change correct version
# ----------------------------------------------------------
def correct_add_function(arg1, arg2):
    """
    Correct version of the above function (pairwise addition).
    """
    arg1_index = 0
    while arg1_index < len(arg1):
        addend = arg2[arg1_index] if arg1_index < len(arg2) else 0
        arg_2_sum = 0
        for j, v in enumerate(arg2):
            if j == arg1_index:
                arg_2_sum = arg1[arg1_index] + v
                break
        else:
            arg_2_sum = arg1[arg1_index] + 0
        arg1[arg1_index] = arg_2_sum
        arg1_index += 1
    return arg1


# ----------------------------------------------------------
# Helper predicates
# ----------------------------------------------------------
def _all_ints(lst): 
    return all(isinstance(x, int) for x in lst)

def _all_strs(lst): 
    return all(isinstance(x, str) for x in lst)


# ----------------------------------------------------------
# 2.b — Exception-handling version
# ----------------------------------------------------------
def exception_add_function(arg1, arg2):
    """
    Checks for invalid input types and returns descriptive error message.
    """
    try:
        # numeric branch (correct logic)
        if _all_ints(arg1) and _all_ints(arg2):
            return correct_add_function(arg1[:], arg2[:])

        # string branch (original concatenation)
        if _all_strs(arg1) and _all_strs(arg2):
            a1 = arg1[:]
            arg1_index = 0
            while arg1_index < len(a1):
                arg_2_sum = ''
                for arg2_elements in arg2:
                    arg_2_sum += arg2_elements
                a1[arg1_index] = a1[arg1_index] + str(arg_2_sum)
                arg1_index += 1
            return a1

        # otherwise produce error message
        expect_ints = any(isinstance(x, int) for x in arg1 + arg2)
        if expect_ints:
            for n, v in enumerate(arg1):
                if not isinstance(v, int):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, int):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."
        else:
            for n, v in enumerate(arg1):
                if not isinstance(v, str):
                    return f"Your input argument 1 at element {n} is not of the expected type. Please change this and rerun."
            for n, v in enumerate(arg2):
                if not isinstance(v, str):
                    return f"Your input argument 2 at element {n} is not of the expected type. Please change this and rerun."

        return "Invalid inputs."
    except Exception:
        return "An unexpected error occurred while processing your inputs."


# ----------------------------------------------------------
# 2.c — Correction wrapper (coerces mixed input to strings)
# ----------------------------------------------------------
def correction_add_function(arg1, arg2):
    """
    Coerces mixed inputs so wrong_add_function’s string section runs cleanly.
    """
    try:
        if any(isinstance(x, str) for x in arg1 + arg2):
            a1 = [str(x) for x in arg1]
            a2 = [str(x) for x in arg2]
            return wrong_add_function(a1, a2)
        return wrong_add_function(arg1[:], arg2[:])
    except Exception:
        return "Could not coerce inputs for string processing."


# ----------------------------------------------------------
# Assign to question variables
# ----------------------------------------------------------
q_1a = wrong_add_function
q_1b = correct_add_function
q_2b = exception_add_function
q_2c = correction_add_function


# ----------------------------------------------------------
# Example variable inputs
# ----------------------------------------------------------
demo_arg1 = [1, 2, 3]
demo_arg2 = [1, 1, 1]
demo_arg_str_1 = ['1', '2', '3']
demo_arg_str_2 = ['1', '1', 1]


# ----------------------------------------------------------
# Function calls (prints & returns)
# ----------------------------------------------------------
print("\n--- 1.a wrong_add_function ---")
result_1a = q_1a(demo_arg1[:], demo_arg2[:])
print("Output (1.a):", result_1a)

print("\n--- 1.b correct_add_function ---")
result_1b = q_1b(demo_arg1[:], demo_arg2[:])
print("Output (1.b):", result_1b)

print("\n--- 2.b exception_add_function ---")
result_2b_valid = q_2b(demo_arg1[:], demo_arg2[:])
result_2b_error = q_2b(['1', 2, 3], [1, 1, 1])
print("Output (2.b valid):", result_2b_valid)
print("Output (2.b error):", result_2b_error)

print("\n--- 2.c correction_add_function ---")
result_2c = q_2c(demo_arg_str_1[:], demo_arg_str_2[:])
print("Output (2.c):", result_2c)


--- 1.a wrong_add_function ---
We are making an error in the loop at index 0. The correct answer is supposed to be: 2
[DEBUG] Buggy will sum [2, 2, 2] -> 6
We are making an error in the loop at index 1. The correct answer is supposed to be: 3
[DEBUG] Buggy will sum [3, 3, 3] -> 9
We are making an error in the loop at index 2. The correct answer is supposed to be: 4
[DEBUG] Buggy will sum [4, 4, 4] -> 12
Output (1.a): [6, 9, 12]

--- 1.b correct_add_function ---
Output (1.b): [2, 3, 4]

--- 2.b exception_add_function ---
Output (2.b valid): [2, 3, 4]
Output (2.b error): Your input argument 1 at element 0 is not of the expected type. Please change this and rerun.

--- 2.c correction_add_function ---
We are making an error in the loop at index 0. The correct answer is supposed to be: 11
Output (2.c): Could not coerce inputs for string processing.
